In [1]:
import pandas as pd
import boto3
import re
import io

/tmp/ipykernel_4967/2994570450.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Read Data From S3 Bucket

In [2]:
s3_client = boto3.client('s3')

In [3]:
obj = s3_client.get_object(Bucket="storetransactiondemo1", Key="dirty_store_transactions.csv")

In [4]:
df = pd.read_csv(obj['Body'])

In [5]:
df.head()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York(,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York+,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York!,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


# Transform and Clean Data

In [6]:
def clean_store_location(st_loc):
        return re.sub(r'[^\w\s]', '', st_loc).strip()

In [7]:
df['STORE_LOCATION'] = df['STORE_LOCATION'].map(lambda x: clean_store_location(x))

In [8]:
df.head()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York,Furniture,72619323C,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682B,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263T,$85,$51,$2.55,$82.45,2019-11-26


In [9]:
def clean_product_id(pd_id):
        matches = re.findall(r'\d+', pd_id)
        if matches:
            return matches[0]
        return pd_id

In [10]:
df['PRODUCT_ID'] = df['PRODUCT_ID'].map(lambda x: clean_product_id(x))

In [11]:
df.head()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York,Electronics,12254943,$31,$20.77,$1.86,$29.14,2019-11-26
1,YR7220,New York,Furniture,72619323,$15,$9.75,$1.5,$13.5,2019-11-26
2,YR7220,New York,Electronics,34161682,$88,$62.48,$4.4,$83.6,2019-11-26
3,YR7220,New York,Kitchen,79411621,$91,$58.24,$3.64,$87.36,2019-11-26
4,YR7220,New York,Fashion,39520263,$85,$51,$2.55,$82.45,2019-11-26


In [12]:
def remove_dollar(amount):
    return float(amount.replace('$', ''))

In [13]:
for to_clean in ['MRP', 'CP', 'DISCOUNT', 'SP']:
        df[to_clean] = df[to_clean].map(lambda x: remove_dollar(x))

In [14]:
df.head()

,STORE_ID,STORE_LOCATION,PRODUCT_CATEGORY,PRODUCT_ID,MRP,CP,DISCOUNT,SP,Date
0,YR7220,New York,Electronics,12254943,31.0,20.77,1.86,29.14,2019-11-26
1,YR7220,New York,Furniture,72619323,15.0,9.75,1.50,13.50,2019-11-26
2,YR7220,New York,Electronics,34161682,88.0,62.48,4.40,83.60,2019-11-26
3,YR7220,New York,Kitchen,79411621,91.0,58.24,3.64,87.36,2019-11-26
4,YR7220,New York,Fashion,39520263,85.0,51.00,2.55,82.45,2019-11-26


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37853 entries, 0 to 37852
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   STORE_ID          37853 non-null  object 
 1   STORE_LOCATION    37853 non-null  object 
 2   PRODUCT_CATEGORY  37853 non-null  object 
 3   PRODUCT_ID        37853 non-null  object 
 4   MRP               37853 non-null  float64
 5   CP                37853 non-null  float64
 6   DISCOUNT          37853 non-null  float64
 7   SP                37853 non-null  float64
 8   Date              37853 non-null  object 
dtypes: float64(4), object(5)
memory usage: 2.6+ MB


# Save Clean Data to AWS RDS MySQL